<a href="https://colab.research.google.com/github/BerryGIT-ME/NLP/blob/feature%2Fdev-chatbots/chat_bot_with_context_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# We creat a basic context aware chatbot with this notebook.

We make use of langchain and llama-index embdeddings

Best performance is achieved with a GPU.

We make use of openAI embedding models to create the embeddings which are used to provide context to the language model.

An open sourced llm model is used - this way the only cost to run the notebook is the cost associated with creating the embeddings

In [1]:
# Check if a GPU is available
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
# Install dependencies
! pip install -q langchain transformers sentence_transformers llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.4/515.4 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 94.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 10.3 MB/s eta 0:00:00
  

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.llms.base import LLM
from transformers import pipeline
from llama_index import (
    SimpleDirectoryReader,
    LangchainEmbedding,
    GPTSimpleVectorIndex,
    PromptHelper,
    LLMPredictor,
    ServiceContext,
    StorageContext,
    readers,
    load_index_from_storage,
    Document
    )
import torch
import openai

In [ ]:
openai.api_key = 'Your openai api key'

In [ ]:
# langchain wrapper for a custom llm
class customLLM(LLM):
    model_name = "google/flan-t5-large"
    pipeline = pipeline("text2text-generation", model=model_name, device=0, model_kwargs={"torch_dtype":torch.bfloat16})

    def _call(self, prompt, stop=None):
        return self.pipeline(prompt, max_length=9999)[0]["generated_text"]

    @property
    def _identifying_params(self):
        return {"name_of_model": self.model_name}

    @property
    def _llm_type(self):
        return "custom"


llm_predictor = LLMPredictor(llm=customLLM())



In [ ]:
hfemb = HuggingFaceEmbeddings()
embed_model = LangchainEmbedding(hfemb)

In [ ]:
text1 = """
Enabling Mobile Payment and Digital Finance in Africa
QOS APIs allow companies of all sizes — from startups to large enterprises — to accept payments, make transfers, receive payouts, and manage their businesses online
Do you have a sandbox where I can test your API?

Yes, we have a sandbox for you to test the API. Check the developer page of our website for more information on the sandbox. Create an account and to start accepting payment.

How do I receive the money paid by my customers?

You will be settled within 24 hours after a successful payment and you can withdraw your money to you registered account at any time after settlement.

Do I need a Merchant Mobile Money Account?

For mobile money payout, it is advisable to have a merchant account for payout because you can link the merchant account to your bank account. However, payout can be done to any mobile money account number Visit the operator’s mobile money office to register for a merchant account if you don’t already have one.

How long does it take for me to get my money (settlement)?

Settlement to your QOS account is done within 48 hours after a successful payment.

How much do you charge to integrate to the API?

Integration with our API is free. You can create an account here.

How much do you charge to develop a USSD menu?

The price for developing your USSD menu depends on the complexity of your customer journey. We will be glad to discuss with you to find a suitable solution for your use case.

How much do you charge to onboard on Mobile Money USSD?

It is free to onboard on QOS position on the mobile money USSD menu.

Do you charge fee/commission on collection?

There is 1.7% commission on every successful payment transaction you receive.

Do you charge for transfers (disbursement)?

There is 1% on every successful transfer to other mobile money accounts.

Do you charge for payout to my account (settlement)?

Payout into your account is free.

Does integrating your API give access to other countries?

At this moment, our API gives you access to Benin, Togo and Cote D’Ivoire. As we add more countries, we shall notify you accordingly

Do you offer transactions volume discounts ?

Yes, high volume customers can benefit from a reduced commission on transaction volumes. Please contact sales for more information.

Are you able to offer flat rate per transactions?

Yeah, we offer flat rate per transaction to businesses such Microfinance. In this scenario, we charge a flat fee per successful transaction. Please contact sales to discuss further.
"""

In [ ]:
# To load from a stored text file uncomment the next line of code
#documents = SimpleDirectoryReader('data').load_data()

cdocuments = [Document(t) for t in [text1]]

In [ ]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)

## Create new index from documents



In [ ]:
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model)
index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)

In [ ]:
folder_path = 'index_store'
index.storage_context.persist(persist_dir=folder_path)

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("what are the articles about?")
print(response)

In [ ]:
response = query_engine.query("what offers can I get from qos?")
print(response)


In [ ]:
response = query_engine.query("How are customers charged for the use of qos apis")
print(response)

In [ ]:
response = query_engine.query("what about commissions?")
print(response)

## Load previously saved index from hard drive

In [ ]:
folder_path = 'index_store'
storage_context = StorageContext.from_defaults(persist_dir=folder_path)
index2 = load_index_from_storage(storage_context, service_context=service_context)

In [ ]:
query_engine2 = index2.as_query_engine()
response = query_engine2.query("what are the articles about?")
print(response)

QOS APIs
